In [1]:
import pandas as pd
import json
import torch
import random
from pathlib import Path
from pandas import json_normalize
from transformers import AutoTokenizer, AutoModel, AdamW
from helpers import tokenize_and_format, flat_accuracy

c:\Users\amado\anaconda3\envs\cse1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [2]:
files = {
    "train": Path("LegalSupport", "train.jsonl"),
    "dev": Path("LegalSupport", "dev.jsonl"),
    "test": Path("LegalSupport", "test.jsonl")
}

In [3]:
df = pd.read_json(files["train"], lines=True).explode("label")
df.head()

,context,citation_a,citation_b,case_id,label
0,Witnesses are absolutely immune for the testim...,"{'signal': 'see also', 'identifier': '25 F.3d ...","{'signal': 'but see', 'identifier': '17 F.3d 1...",11529769,a
1,We exercise de novo review over a district cou...,"{'signal': 'see', 'identifier': '489 F.2d 661,...","{'signal': 'see also', 'identifier': '237 F.3d...",3821705,a
2,"We do not, at this time, reach the merits of t...","{'signal': 'no signal', 'identifier': '859 F.2...","{'signal': 'see', 'identifier': '884 F.2d 879,...",11644198,a
3,"We do not, at this time, reach the merits of t...","{'signal': 'no signal', 'identifier': None, 'p...","{'signal': 'see', 'identifier': '884 F.2d 879,...",11644198,a
4,"We do not, at this time, reach the merits of t...","{'signal': 'no signal', 'identifier': None, 'p...","{'signal': 'see', 'identifier': '884 F.2d 879,...",11644198,a


In [4]:
df["citation"] = df.apply(lambda a: a["citation_a"] if a["label"] == "a" else a["citation_b"], axis=1)
df = df.drop(["citation_a", "citation_b", "label"], axis=1)

In [5]:
df.loc[0, "citation"]

{'signal': 'see also',
 'identifier': '25 F.3d 83, 83',
 'parenthetical': 'holding that while witnesses enjoy absolute immunity for their actions in testifying they are not immune for extra-judicial actions such as an alleged conspiracy to present false testimony',
 'sentence': 'Malachowski, 787 F.2d at 712; see also Dory, 25 F.3d at 83 (holding that while witnesses enjoy absolute immunity for their actions in testifying they are not immune for extra-judicial actions such as an alleged conspiracy to present false testimony).'}

In [6]:
df["case_id"].unique()

array([11529769,  3821705, 11644198, ...,  3642473,   246165,   243105],
      dtype=int64)

In [7]:
df.iloc[0,-1]

{'signal': 'see also',
 'identifier': '25 F.3d 83, 83',
 'parenthetical': 'holding that while witnesses enjoy absolute immunity for their actions in testifying they are not immune for extra-judicial actions such as an alleged conspiracy to present false testimony',
 'sentence': 'Malachowski, 787 F.2d at 712; see also Dory, 25 F.3d at 83 (holding that while witnesses enjoy absolute immunity for their actions in testifying they are not immune for extra-judicial actions such as an alleged conspiracy to present false testimony).'}

In [8]:
df["signal"] = df.apply(lambda a: a["citation"]["signal"], axis=1)
df["identifier"] = df.apply(lambda a: a["citation"]["identifier"], axis=1)
df["parenthetical"] = df.apply(lambda a: a["citation"]["parenthetical"], axis=1)
df["sentence"] = df.apply(lambda a: a["citation"]["sentence"], axis=1)
df = df.drop(["citation"], axis=1)
df.head()

,context,case_id,signal,identifier,parenthetical,sentence
0,Witnesses are absolutely immune for the testim...,11529769,see also,"25 F.3d 83, 83",holding that while witnesses enjoy absolute im...,"Malachowski, 787 F.2d at 712; see also Dory, 2..."
1,We exercise de novo review over a district cou...,3821705,see,"489 F.2d 661, 662-63",conducting a de novo review in determining the...,"See Barrows v. Barrows, 489 F.2d 661, 662-63 (..."
2,"We do not, at this time, reach the merits of t...",11644198,no signal,"859 F.2d 842, 844",recognizing federal court's duty to determine ...,"Auto. Ass’n, 859 F.2d 842, 844 (10th Cir. 1988..."
3,"We do not, at this time, reach the merits of t...",11644198,no signal,None,recognizing federal court's duty to determine ...,"Auto. Ass’n, 859 F.2d 842, 844 (10th Cir. 1988..."
4,"We do not, at this time, reach the merits of t...",11644198,no signal,None,recognizing federal court's duty to determine ...,"Auto. Ass’n, 859 F.2d 842, 844 (10th Cir. 1988..."


In [9]:
train_df = df[["context", "parenthetical"]]
train_df = train_df.rename(columns={"context": "text", "parenthetical": "label"})
train_df.head()

,text,label
0,Witnesses are absolutely immune for the testim...,holding that while witnesses enjoy absolute im...
1,We exercise de novo review over a district cou...,conducting a de novo review in determining the...
2,"We do not, at this time, reach the merits of t...",recognizing federal court's duty to determine ...
3,"We do not, at this time, reach the merits of t...",recognizing federal court's duty to determine ...
4,"We do not, at this time, reach the merits of t...",recognizing federal court's duty to determine ...


In [10]:
train_df.to_csv("prel_train_df.csv")